In [1]:
    #IMPORTING DEPENDENCIES

In [2]:
pip install Keras-Preprocessing


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import keras
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk

2024-05-09 19:25:41.938934: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 19:25:41.939032: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 19:25:42.059623: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
r = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
r

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [5]:
#DATA PREPROCESSING

In [6]:
r['review'] = r['review'].str.replace(r'<[^<>]*>', '')
r['review'] = r['review'].replace(to_replace=r'^https?:\/\/.*[\r\n]*',value='')
r['review'] = r['review'].str.lower()

In [7]:
d = pd.DataFrame({'strings': ['a#bc1!', 'a(b$c']})

d.strings.str.replace('[^a-zA-Z0-9]', '')

0    a#bc1!
1     a(b$c
Name: strings, dtype: object

In [8]:
r

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive
...,...,...
49995,i thought this movie did a down right good job...,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,i am a catholic taught in parochial elementary...,negative
49998,i'm going to have to disagree with the previou...,negative


In [9]:
#RECURRING WORDS

In [10]:

# for i in range(len(r['review'])):
#     pd.Series(" ".join(r['review']).split()).value_counts()>100

In [11]:
rem_stopwords = ['it.', 'could', 'made', '-', 'get', 'also', 'would', 'get', 'seen', 'go', 'two', 'one']

In [12]:
#REMOVING STOPWORDS

In [13]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
r['review'] = r['review'].apply(lambda x: " ".join([word for word in x.split() if word not in stop_words]))
r['review'] = r['review'].apply(lambda x: " ".join([word for word in x.split() if word not in rem_stopwords]))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
#TOKENIZATION

In [15]:
# from pandarallel import pandarallel

# pandarallel.initialize(progress_bar=True)
# r['tokenized'] = r['review'].parallel_apply(lambda x: nltk.word_tokenize(x))

In [16]:
# r['review'].apply(lambda x: len(x.split()))

In [17]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [18]:
from textblob import Word
r['lemmatized'] = r['review'].apply(lambda x: " ".join(Word(word).lemmatize() for word in str(x).split()))

In [19]:
recuuring_words = pd.Series(" ".join(r['review']).split()).value_counts()

In [20]:
(recuuring_words>10).value_counts()     #choosing vocab_size

count
False    350856
True      39899
Name: count, dtype: int64

In [21]:
r

,review,sentiment,lemmatized
0,reviewers mentioned watching 1 oz episode hook...,positive,reviewer mentioned watching 1 oz episode hooke...
1,wonderful little production. <br /><br />the f...,positive,wonderful little production. <br /><br />the f...
2,thought wonderful way spend time hot summer we...,positive,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...,negative,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu...",positive,"petter mattei's ""love time money"" visually stu..."
...,...,...,...
49995,thought movie right good job. creative origina...,positive,thought movie right good job. creative origina...
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative,"bad plot, bad dialogue, bad acting, idiotic di..."
49997,catholic taught parochial elementary schools n...,negative,catholic taught parochial elementary school nu...
49998,i'm going disagree previous comment side malti...,negative,i'm going disagree previous comment side malti...


In [22]:
# pos = 0
# for i in range(r.shape[0]):
#                if r.iloc[i]['sentiment'] == 'positive':
#                     pos = pos+1
# neg = r.shape[0]-pos
# posper = pos/r.shape[0]*100
# negper = neg/r.shape[0]*100
# print(posper)
# print(negper)

In [23]:
#ENCODING

In [24]:
reviews = r['lemmatized'].values
labels = r['sentiment'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)    #converting positive  to 1 and negative to 0

In [25]:
#SPLITTING TRAINING AND TESTING DATA

In [26]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, stratify= encoded_labels)

In [27]:
#HYPERPARAMETERS OF MODEL

In [28]:
vocab_size = 10000
tok = ''
embedding_dim = 100
max_length = 300
padding_type = 'post' 
trunc_type = 'post'

In [29]:
#TOKENIZE SENTENCES

In [30]:
tokenize = Tokenizer(num_words = vocab_size, oov_token =tok)
tokenize.fit_on_texts(train_sentences)
word_index = tokenize.word_index

In [31]:
#CONVERTING TRAIN DATASET TO SEQUENCES AND PAD SEQUENCES

In [32]:
train_sequences = tokenize.texts_to_sequences(train_sentences)
train_padding = pad_sequences(train_sequences, padding = 'post', maxlen = max_length)

In [33]:
sequence_length = max(len(sentence.split()) for sentence in train_sentences)


In [34]:
print(sequence_length)

1436


In [35]:
#CONVERTING TEST DATASET TO SEQUENCES AND PAD SEQUENCES

In [36]:
test_sequences = tokenize.texts_to_sequences(test_sentences)
test_padding = pad_sequences(test_sequences, padding = 'post', maxlen = max_length)

In [48]:
#BUILDING MODEL
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim),
    keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])
model.build((None, sequence_length))

In [49]:
#Initialisation

In [50]:
#COMPILING

In [56]:
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model with early stopping
history = model.fit(train_padding, train_labels,
                    epochs=n_epochs, verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 1436, 100)      │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 1436, 128)      │        84,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 24)             │         3,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 24)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            25 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,186,417 (4.53 MB)

 Trainable params: 1,186,417 (4.53 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
1055/1055 ━━━━━━━━━━━━━━━━━━━━ 69s 62ms/step - accuracy: 0.9884 - loss: 0.0526 - val_accuracy: 0.8789 - val_loss: 0.6317
Epoch 2/20
1055/1055 ━━━━━━━━━━━━━━━━━━━━ 65s 61ms/step - accuracy: 0.9923 - loss: 0.0391 - val_accuracy: 0.8717 - val_loss: 0.5902
Epoch 3/20
1055/1055 ━━━━━━━━━━━━━━━━━━━━ 65s 62ms/step - accuracy: 0.9942 - loss: 0.0262 - val_accuracy: 0.8744 - val_loss: 0.7471
Epoch 4/20
1055/1055 ━━━━━━━━━━━━━━━━━━━━ 65s 62ms/step - accuracy: 0.9956 - loss: 0.0198 - val_accuracy: 0.8752 - val_loss: 0.7518
Epoch 5/20
1055/1055 ━━━━━━━━━━━━━━━━━━━━ 65s 62ms/step - accuracy: 0.9965 - loss: 0.0163 - val_accuracy: 0.8717 - val_loss: 0.8283


In [57]:
prediction = model.predict(test_padding)


391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step


In [58]:
#LABELNG BASED ON PROBABILITY

In [59]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Your existing code
p_labels = []
for i in prediction:
    if i > 0.5:
        p_labels.append(1)
    elif i == 0.5:
        p_labels.append(2)
    else:
        p_labels.append(0)

# Calculate accuracy
accuracy = accuracy_score(test_labels, p_labels)
print("Accuracy: ", accuracy)

# Calculate precision
precision = precision_score(test_labels, p_labels, average='weighted')
print("Precision: ", precision)

# Calculate recall
recall = recall_score(test_labels, p_labels, average='weighted')
print("Recall: ", recall)

# Calculate F1 score
f1 = f1_score(test_labels, p_labels, average='weighted')
print("F1 Score: ", f1)

Accuracy:  0.87992
Precision:  0.8806250857428336
Recall:  0.87992
F1 Score:  0.879864363937529


In [60]:
sentence = ["The movie was best",
           "The plot of movie was terrible",
           "The actors did a very good job",
           'beautiful movie. loved it',
           "it was boring",
           "VFX were a spectacle to see",
           "i loved the movie",
           " It stinks"]
sequences = tokenize.texts_to_sequences(sentence)
padding = pad_sequences(sequences, padding = 'post', maxlen = max_length)
prediction = model.predict(padding)
p_labels = []
for i in prediction:
    if i > 0.5:
        p_labels.append(1)
    elif i==0.5:
        p_labels.append(2)
    else:
        p_labels.append(0)
for i in range (len(sentence)):
    if p_labels[i]==1:
        s= 'Positive'
    elif p_labels[i]==2:
        s='Neutral'
    else:
        s='Negative'
    print("Sentiment"+str(i)+" is: ",s)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Sentiment0 is:  Positive
Sentiment1 is:  Negative
Sentiment2 is:  Negative
Sentiment3 is:  Positive
Sentiment4 is:  Negative
Sentiment5 is:  Positive
Sentiment6 is:  Positive
Sentiment7 is:  Negative


In [61]:
model.save('senty.keras')